In [ ]:
!pip install -q flwr[simulation] torch torchvision

     |████████████████████████████████| 121 kB 7.9 MB/s 
     |████████████████████████████████| 60.2 MB 87 kB/s 
     |████████████████████████████████| 4.1 MB 42.7 MB/s 
     |████████████████████████████████| 8.8 MB 29.9 MB/s 
     |████████████████████████████████| 128 kB 62.5 MB/s 
     |████████████████████████████████| 57 kB 3.1 MB/s 
     |████████████████████████████████| 90 kB 5.6 MB/s 
     |████████████████████████████████| 3.0 MB 49.1 MB/s 
     |████████████████████████████████| 201 kB 58.7 MB/s 
     |████████████████████████████████| 280 kB 54.0 MB/s 
     |████████████████████████████████| 58 kB 999 kB/s 
     |████████████████████████████████| 468 kB 52.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.48.2 requires grpcio>=1.48.2, but you have grpcio 1.43.0 which is incompatible.
google-cloud-bigquery 3.3.6 requires grpc

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cpu")  
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.12.1+cu113 and Flower 1.1.0


In [ ]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):

    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10 
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:

params = get_parameters(Net())

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  
    strategy=strategy,
)

INFO flower 2022-12-01 20:58:34,720 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-01 20:58:37,296	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-01 20:58:39,038 | app.py:174 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3781043404.0, 'node:172.28.0.12': 1.0, 'memory': 7562086811.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3781043404.0, 'node:172.28.0.12': 1.0, 'memory': 7562086811.0}
INFO flower 2022-12-01 20:58:39,052 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-12-01 20:58:39,065 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-12-01 20:58:39,078 | server.py:88 | Evaluat

(launch_and_fit pid=274) [Client 3] fit, config: {}
(launch_and_fit pid=275) [Client 8] fit, config: {}
(launch_and_fit pid=274) Epoch 1: train loss 0.06570924818515778, accuracy 0.21444444444444444
(launch_and_fit pid=275) Epoch 1: train loss 0.06481186300516129, accuracy 0.22244444444444444
(launch_and_fit pid=274) [Client 6] fit, config: {}


DEBUG flower 2022-12-01 20:58:56,580 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-01 20:58:56,597 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-12-01 20:58:56,602 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=274) Epoch 1: train loss 0.06517679244279861, accuracy 0.22266666666666668
(launch_and_evaluate pid=274) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=275) [Client 7] evaluate, config: {}


DEBUG flower 2022-12-01 20:59:02,595 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flower:evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-12-01 20:59:02,600 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-01 20:59:02,602 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flower:fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flower 2022-12-01 20:59:33,173 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG:flower:fit_round 3 received 3 results and 0 failures
DEBUG flower 2022-12-01 20:59:33,189 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=274) Epoch 1: train loss 0.051758743822574615, accuracy 0.39555555555555555
(launch_and_evaluate pid=274) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=275) [Client 1] evaluate, config: {}


DEBUG flower 2022-12-01 20:59:38,622 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-01 20:59:38,628 | server.py:144 | FL finished in 59.523762938999994
INFO:flower:FL finished in 59.523762938999994
INFO flower 2022-12-01 20:59:38,639 | app.py:192 | app_fit: losses_distributed [(1, 0.0630527941385905), (2, 0.05505398464202881), (3, 0.05101317604382833)]
INFO:flower:app_fit: losses_distributed [(1, 0.0630527941385905), (2, 0.05505398464202881), (3, 0.05101317604382833)]
INFO flower 2022-12-01 20:59:38,642 | app.py:193 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-12-01 20:59:38,645 | app.py:194 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-12-01 20:59:38,647 | app.py:195 | app_fit: metrics_centralized {}
INFO:flower:app_fit: metrics_centralized {}


(launch_and_evaluate pid=274) [Client 6] evaluate, config: {}


History (loss, distributed):
	round 1: 0.0630527941385905
	round 2: 0.05505398464202881
	round 3: 0.05101317604382833

In [ ]:

strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  
    strategy=strategy,
)

Streaming output truncated to the last 5000 lines.
KeyError: 'server_round'
DEBUG:flower:ray::launch_and_fit() (pid=2859, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/app.py", line 252, in _fit
    results = self.numpy_client.fit(parameters, ins.config)  # type: ignore
  File "<ipython-input-11-1d00d3d2ae0b>", line 14, in fit
KeyError: 'server_round'
DEBUG flower 2022-12-01 22:31:09,460 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=2859, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.

(launch_and_evaluate pid=2860) [Client 220] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 79] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 139] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 567] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 302] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 29] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 469] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 714] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 537] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 788] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 465] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 468] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 653] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 46] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 254] evaluate, config: {}
(launch_and_evaluate pid=285

DEBUG flower 2022-12-01 22:47:57,749 | server.py:179 | evaluate_round 1 received 300 results and 0 failures
DEBUG:flower:evaluate_round 1 received 300 results and 0 failures
WARNING flower 2022-12-01 22:47:57,753 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-01 22:47:57,758 | server.py:215 | fit_round 2: strategy sampled 300 clients (out of 1000)
DEBUG:flower:fit_round 2: strategy sampled 300 clients (out of 1000)


(launch_and_evaluate pid=2859) [Client 779] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 78] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 500] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 898] evaluate, config: {}


Streaming output truncated to the last 5000 lines.
KeyError: 'server_round'
DEBUG:flower:ray::launch_and_fit() (pid=2859, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/app.py", line 252, in _fit
    results = self.numpy_client.fit(parameters, ins.config)  # type: ignore
  File "<ipython-input-11-1d00d3d2ae0b>", line 14, in fit
KeyError: 'server_round'
DEBUG flower 2022-12-01 22:48:43,880 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=2860, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.

(launch_and_evaluate pid=2859) [Client 766] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 292] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 393] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 650] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 900] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 204] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 43] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 821] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 336] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 566] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 222] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 330] evaluate, config: {}
(launch_and_evaluate pid=2860) [Client 291] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 214] evaluate, config: {}
(launch_and_evaluate pid=2859) [Client 368] evaluate, config: {}
(launch_and_evaluate pid=2

DEBUG flower 2022-12-01 23:05:27,689 | server.py:179 | evaluate_round 2 received 300 results and 0 failures
DEBUG:flower:evaluate_round 2 received 300 results and 0 failures
DEBUG flower 2022-12-01 23:05:27,704 | server.py:215 | fit_round 3: strategy sampled 300 clients (out of 1000)
DEBUG:flower:fit_round 3: strategy sampled 300 clients (out of 1000)


(launch_and_evaluate pid=2860) [Client 494] evaluate, config: {}


DEBUG flower 2022-12-01 23:05:36,969 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=2860, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/app.py", line 252, in _fit
    results = self.numpy_client.fit(parameters, ins.config)  # type: ignore
  File "<ipython-input-11-1d00d3d2ae0b>", line 14, in fit
KeyError: 'server_round'
DEBUG:flower:ray::launch_and_fit() (pid=2860, ip=172.28.0.12)
  File "/usr/local/lib/python3.8/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "/usr/local/lib/python3.8/dist-packages/flwr/client/client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "/usr

In [ ]:

def evaluate(
    server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  
    strategy=strategy,
)

INFO flower 2022-12-01 21:09:17,004 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-01 21:09:21,223	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-01 21:09:22,754 | app.py:174 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3850508697.0, 'CPU': 2.0, 'memory': 7701017396.0, 'node:172.28.0.12': 1.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'object_store_memory': 3850508697.0, 'CPU': 2.0, 'memory': 7701017396.0, 'node:172.28.0.12': 1.0}
INFO flower 2022-12-01 21:09:22,765 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-12-01 21:09:22,773 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-12-01 21:09:22,798 | server.py:88 | Evaluat

Server-side evaluation loss 0.07372414588928222 / accuracy 0.1
(launch_and_fit pid=871) [Client 1] fit, config: {}
(launch_and_fit pid=870) [Client 0] fit, config: {}
(launch_and_fit pid=871) Epoch 1: train loss 0.06471272557973862, accuracy 0.23666666666666666
(launch_and_fit pid=870) Epoch 1: train loss 0.06466713547706604, accuracy 0.2408888888888889
(launch_and_fit pid=871) [Client 8] fit, config: {}


DEBUG flower 2022-12-01 21:09:38,064 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-01 21:09:38,095 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=871) Epoch 1: train loss 0.06452125310897827, accuracy 0.23466666666666666


INFO flower 2022-12-01 21:09:38,878 | server.py:116 | fit progress: (1, 0.06180789756774902, {'accuracy': 0.294}, 15.660324368000033)
INFO:flower:fit progress: (1, 0.06180789756774902, {'accuracy': 0.294}, 15.660324368000033)
DEBUG flower 2022-12-01 21:09:38,883 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

INFO flower 2022-12-01 21:10:15,267 | server.py:116 | fit progress: (3, 0.053113158464431764, {'accuracy': 0.394}, 52.048721419)
INFO:flower:fit progress: (3, 0.053113158464431764, {'accuracy': 0.394}, 52.048721419)
DEBUG flower 2022-12-01 21:10:15,273 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.053113158464431764 / accuracy 0.394
(launch_and_evaluate pid=870) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=871) [Client 6] evaluate, config: {}


DEBUG flower 2022-12-01 21:10:20,436 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-01 21:10:20,440 | server.py:144 | FL finished in 57.22195833000001
INFO:flower:FL finished in 57.22195833000001
INFO flower 2022-12-01 21:10:20,443 | app.py:192 | app_fit: losses_distributed [(1, 0.06180546085039775), (2, 0.056890295664469404), (3, 0.052713688135147097)]
INFO:flower:app_fit: losses_distributed [(1, 0.06180546085039775), (2, 0.056890295664469404), (3, 0.052713688135147097)]
INFO flower 2022-12-01 21:10:20,445 | app.py:193 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-12-01 21:10:20,448 | app.py:194 | app_fit: losses_centralized [(0, 0.07372414588928222), (1, 0.06180789756774902), (2, 0.0563735728263855), (3, 0.053113158464431764)]
INFO:flower:app_fit: losses_centralized [(0, 0.07372414588928222), (1, 0.06180789756774902), (2, 0.0563735

(launch_and_evaluate pid=871) [Client 4] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06180546085039775
	round 2: 0.056890295664469404
	round 3: 0.052713688135147097
History (loss, centralized):
	round 0: 0.07372414588928222
	round 1: 0.06180789756774902
	round 2: 0.0563735728263855
	round 3: 0.053113158464431764
History (metrics, centralized):
{'accuracy': [(0, 0.1), (1, 0.294), (2, 0.348), (3, 0.394)]}

In [ ]:
def fit_config(server_round: int):
    
    config = {
        "server_round": server_round,  
        "local_epochs": 1 if server_round < 2 else 2,  # 
    }
    return config

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  
    strategy=strategy,
)

INFO flower 2022-12-01 21:10:27,343 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-01 21:10:32,373	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-01 21:10:33,858 | app.py:174 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7699434702.0, 'object_store_memory': 3849717350.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7699434702.0, 'object_store_memory': 3849717350.0}
INFO flower 2022-12-01 21:10:33,863 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-12-01 21:10:33,866 | server.py:266 | Using initial parameters provided by strategy
INFO:flower:Using initial parameters provided by strategy
INFO flower 2022-12-01 21:10:33,877 | server.py:88 | Evaluat

Server-side evaluation loss 0.07359691905975342 / accuracy 0.134
(launch_and_fit pid=1109) [Client 3, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1110) [Client 8, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1109) Epoch 1: train loss 0.06529038399457932, accuracy 0.21733333333333332
(launch_and_fit pid=1110) Epoch 1: train loss 0.06549897789955139, accuracy 0.22155555555555556
(launch_and_fit pid=1109) [Client 6, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flower 2022-12-01 21:10:49,256 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flower:fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-01 21:10:49,273 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flower 2022-12-01 21:10:49,490 | server.py:116 | fit progress: (1, 0.06458748602867126, {'accuracy': 0.234}, 15.21001259000002)
INFO:flower:fit progress: (1, 0.06458748602867126, {'accuracy': 0.234}, 15.21001259000002)
DEBUG flower 2022-12-01 21:10:49,497 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1109) Epoch 1: train loss 0.06633482873439789, accuracy 0.214
Server-side evaluation loss 0.06458748602867126 / accuracy 0.234
(launch_and_evaluate pid=1109) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=1110) [Client 1] evaluate, config: {}


DEBUG flower 2022-12-01 21:10:54,937 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flower:evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-12-01 21:10:54,940 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-01 21:10:54,943 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flower:fit_round 2: strategy sampled 3 clients (out of 10)


...

INFO flower 2022-12-01 21:11:48,887 | server.py:116 | fit progress: (3, 0.04963145303726196, {'accuracy': 0.416}, 74.60693754199997)
INFO:flower:fit progress: (3, 0.04963145303726196, {'accuracy': 0.416}, 74.60693754199997)
DEBUG flower 2022-12-01 21:11:48,896 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flower:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04963145303726196 / accuracy 0.416
(launch_and_evaluate pid=1109) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=1110) [Client 1] evaluate, config: {}


DEBUG flower 2022-12-01 21:11:54,270 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flower:evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-01 21:11:54,281 | server.py:144 | FL finished in 80.00106980099997
INFO:flower:FL finished in 80.00106980099997
INFO flower 2022-12-01 21:11:54,290 | app.py:192 | app_fit: losses_distributed [(1, 0.06396356622378031), (2, 0.05190704719225566), (3, 0.048628463983535763)]
INFO:flower:app_fit: losses_distributed [(1, 0.06396356622378031), (2, 0.05190704719225566), (3, 0.048628463983535763)]
INFO flower 2022-12-01 21:11:54,294 | app.py:193 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-12-01 21:11:54,298 | app.py:194 | app_fit: losses_centralized [(0, 0.07359691905975342), (1, 0.06458748602867126), (2, 0.0539579541683197), (3, 0.04963145303726196)]
INFO:flower:app_fit: losses_centralized [(0, 0.07359691905975342), (1, 0.06458748602867126), (2, 0.0539579541

(launch_and_evaluate pid=1109) [Client 7] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06396356622378031
	round 2: 0.05190704719225566
	round 3: 0.048628463983535763
History (loss, centralized):
	round 0: 0.07359691905975342
	round 1: 0.06458748602867126
	round 2: 0.0539579541683197
	round 3: 0.04963145303726196
History (metrics, centralized):
{'accuracy': [(0, 0.134), (1, 0.234), (2, 0.38), (3, 0.416)]}

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)